## <font color = yellow> A single Sale will combine the first row from the Buys & Sells dfs

In [14]:
library(tibble)

In [15]:
cat("One", "Two", "Three", sep = " | ", fill = TRUE)

One | Two | Three


## <font color = yellow> create_df function () - reads .csv

In [17]:
   
df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
df$sell_cf <- as.integer(df$sell_cf)

ERROR: Error in `$<-.data.frame`(`*tmp*`, sell_cf, value = integer(0)): replacement has 0 rows, data has 41


In [ ]:
head(df, 2)


## <font color = yellow> Create dfs

In [ ]:
df_records <- data.frame()
sale_records <- data.frame() 


In [ ]:
# create dfs from filtered df
buys <- df[df$type == 'Buy', ]  # Filter buys
sells <- df[df$type == 'Sell', ]  # Filter sells
b <- buys[1,]
s <- sells[1,]

## <font color = yellow> Basic updates when a sale occurs


            sale <- s$quan
            bs_ratio <- b$quan / s$quan
            cf_quan <-  b$quan - s$quan

            # assign the cf values - if they exist
            if(b$quan > s$quan) {
                b$cf_quan <- (b$quan - s$quan)        
                b$cf_pps <- b$pps
                b$cf_basis <- cf_quan * b$pps
            }
            b$sale_quan <- b$quan
            b$buy_quan <- s$quan 
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            b$sale <- sale

            s$type <- "SOLD"

            return(b)   



In [ ]:
# stopps

## <font color = yellow> The "working" components needed to accomplish the tasks

In [ ]:
for(i in 1:3) {
    cat("for loop, loop is i: ", i, "\n")
    }

In [ ]:
# update b

update_b <- function() {

        sale <- s$quan
        bs_ratio <- b$quan / s$quan
        cf_quan <-  b$quan - s$quan

        # assign the cf values - if they exist
        if(b$quan > s$quan) {
            b$cf_quan <- (b$quan - s$quan)        
            b$cf_pps <- b$pps
            b$cf_basis <- cf_quan * b$pps
        }
        b$sale_quan <- b$quan
       # b$buy_quan <- s$quan 
        b$buy_quan <- b$quan
        b$buy_pps <-  b$pps
        b$buy_basis <-  b$buy_quan * b$pps

        
        # b$sell_quan <- s$quan    # cannot sell more than the buy_quan
        b$sell_quan <- b$quan
        b$sell_pps <- s$pps
        b$sell_basis <- s$quan * s$pps

        b$profit_loss <- b$sell_basis - b$buy_basis
        # b$sale <- sale

        s$type <- "SOLD"

        return(b)   
}

updated_b <- update_b()
updated_b
    
    
    

### <font color = lime> Works with each sale type:( b > s , s > b, b = s, buys_cf, sells_cf )

In [ ]:
evaluate_sale_type <- function () {
    
    cat(" This function displays the Sale Type based on the quan values from the updated_b data. ", "\n")
    cat(" Since the updated_b file is being used for evalutions, KEEP THINGS SIMPLE.  Don't update buys / sells ", "\n\n")

    if(updated_b$quan > updated_b$sell_quan && updated_b$cf_quan > 0 ) {
        cat("This is a BUY carryforward transaction:  ( b_quan > s_quan ) ", "\n")
        cat("Since this is a BUY carryforward transaction, the carryforward values need to be appended to buys[1, ] position", "\n")
        cat(" T/F, good to go. Run:  snip buys[1, ] and sells[1,] ", "\n")
        cat(" And good to go. Record the updated_b into the permanent sales_record df ", "\n")
        cat(" And then begin a new loop " , "\n\n")
    } else if(updated_b$quan == updated_b$sell_quan && updated_b$cf_quan == 0 ) {
        cat("This is a simple transaction:  (b_quan == s_quan) ", "\n")
        cat(" T/F good to go.  Run snip buys[1,], sells[1,] ", "\n")
        cat(" And good to go. Record the updated_b into the permanent sales_record df ", "\n")
        cat(" And then begin a new loop ", "\n\n")            
    }else if (updated_b$quan < updated_b$sell_quan && updated_b$cf_quan == 0 ) {
        cat("This is a SELL carryforward transaction:  ( b_quan < s_quan ) ", "\n")
        cat("Since this is SELL carryforward transaction, the carryforward values need to be appended to sells[1, ] position", "\n")
        cat(" T/F, good to go. Run:  snip buys[1, ] and sells[1,] ", "\n")
        cat(" And good to go. Record the updated_b into the permanent sales_record df ", "\n")
        cat(" And then begin a new loop " , "\n\n")
        }
        
        cat("May need to examime more Sale Types ... when there is / is not CF values in Buys / Sells from previous transaction ", "\n\n")
    
    }  # fct

evaluate_sale_type()

## <font color = yellow> Delete one row from buys and sells

In [ ]:
nrow(buys)
buys <- buys[-1, ]
sells <- sells[-1, ]
nrow(buys)

## <font color = yellow> Update the sales_records df w/ updated_b

In [ ]:
sale_records <- rbind(sale_records, updated_b)
sale_records
                      

In [ ]:
stopps #  Old code

## <font color = yellow> sell_snip & snips functions () - the snips will parse row 1, then call test_for_cf fct

In [ ]:
#  These functions will delete the first row from the Buys and Sells dfs 
#  concluding the first Sale event

i <- 1


# fct
snips <- function (b, buys, df_records) {
    
    b <- b
    buys <- buys  
    df_records <- df_records
    
    # remove the first row of buys ... is b 
    buys <- buys[-1, ]   
    
    
    # Only if, then call fct that will apend df_records for the c/f values.  Otherwise ... ? 
    if(df_records$cf_quan > 0 ) {
        
        #fct call
        buys <- test_for_cf(b, buys, df_records)        
        return(buys)
        
        }else if( df_records$cf_quan == 0 ) {  
                cat("else if ( df_records$cf_quan == 0 .... ??? ", "\n")
            return(buys)       
         }
    
    #  return(buys)
    
   }  # fct

In [ ]:
# fct
sell_snip <- function(sells) {
    sells <- sells    
    sells <- sells[-1, ]    
    return(sells)
    }


# fct call


# nrow(buys)
# nrow(sells)

# buys <- snips(buys)
# sells <- sell_snip(sells)
# nrow(buys)
# nrow(sells)
# head(buys, 3)

In [ ]:
df_records

In [ ]:
# These are the fcts (in order) to process a Sals, from start to end 

nrow(buys)
nrow(sells)

buys <- snips(b,buys, df_records)
sells <- sell_snip(sells)
nrow(buys)
nrow(sells)
head(buys, 3)

In [ ]:
# fct call: to create vars from the first row of Buy & Sell
b <- row_b(b)   
s <- row_s(s)  
b
s

In [ ]:
df_records

head(buys, 3)
head(sells,3)

In [ ]:
 stopps

# <font color = lime> Loop 1 works when there is a carryforward!

## <font color = yellow> Fresh Start: sequence of functions to process one FiFO Sale (one Buy/ one Sell) 
    
        dfs <- create_dfs() 
        row_b <- function(b) 
        row_s <- function(s) 
        when_cf  when_cf(b, s, df_buy_rows)
        df_records(df_records, b, i) 
        test_for_cf (b,buys, df_records)
        snips <- function (buys) 
        sell_snip <- function(sells) 


In [ ]:
# Call the combo function, will return a list of dfs
dfs <- create_dfs()

# Access the dataframes from returned list
df <- dfs$df
df_records <- dfs$df_records
buys <- dfs$buys
sells <- dfs$sells


In [ ]:
b <- row_b(b)   
s <- row_s(s)   

In [ ]:
df_records <- df_create_records(df_records, b, i)

In [ ]:
df_records <- when_cf(b, s, df_records)

In [ ]:
buys <- snips(b,buys, df_records)
sells <- sell_snip(sells)


In [ ]:
head(df_records)
head(buys,2)
head(sells,2)

In [ ]:
stopps

### <font color = yellow> works! Now combine in a loop

In [ ]:
row_b <- function(b) {
        b <- b
        b <- buys[i,]
        return(b)
    }

row_s <- function(s) {
        s <- s
        s <- sells[i,]
        return(s)
    }
  

# nrow(b)
# nrow(s)


In [ ]:
# Call the combo function, will return a list of dfs
dfs <- create_dfs()

# Access the dataframes from returned list
df <- dfs$df
df_records <- dfs$df_records
buys <- dfs$buys
sells <- dfs$sells

b <- row_b(b)   
s <- row_s(s)  

In [ ]:
for(i in 1:nrow(df[1:1, ])) {
    
    cat("test print ", i, "\n")
    
    b <- row_b(b)   
    s <- row_s(s)  
    
    df_records <- df_create_records(df_records, b, i)
    df_records <- when_cf(b, s, df_records)
    buys <- snips(b,buys, df_records)
    sells <- sell_snip(sells)
    
    }
    

In [ ]:
b
s

In [ ]:
head(df_records)
head(buys,3)
head(sells,3)

In [ ]:
stopps # end of loop 1

## <font color = yellow> Works as expected w/ one Buy - one Sell - one loop
    
#### <font color = yellow> See what happens using the next Buy / Sell - loop 2 (involves the c/f from Sale 1 )
    


    
    

In [ ]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 200)

In [ ]:
row_b <- function(b) {
        b <- b
        b <- buys[i,]
        return(b)
    }

row_s <- function(s) {
        s <- s
        s <- sells[i,]
        return(s)
    }
  

# nrow(b)
# nrow(s)


In [ ]:
b
s


In [ ]:
# Call the combo function, will return a list of dfs
dfs <- create_dfs()

# Access the dataframes from returned list
df <- dfs$df
# df_records <- dfs$df_records
buys <- dfs$buys
sells <- dfs$sells

b <- row_b(b)   
s <- row_s(s) 
df_records <- b

In [ ]:
b
s

In [ ]:
for(i in 1:nrow(df[1:2, ])) {
    
    cat("test print ", i, "\n")
    
    b <- buys[i,]
    s <- row_s(s)  
    b
    s
    # df_records <- df_create_records(df_records, b, i)
    df_records <- when_cf(b, s, df_records)
    buys <- snips(b,buys, df_records)
    sells <- sell_snip(sells)
    
    df_records[i,]
    # print(buys[i, ])
    # print(sells[i, ])
    cat("newline ", "\n\n")
    }
    

In [ ]:
head(df_records)

cat("--------------------------------------------------------", "\\n\n\n")
cat("--------------------------------------------------------", "\\n\n\n")
head(buys,3)
head(sells,3)